In [4]:
import numpy as np

In [5]:
# useful matrices
Identity_22 = np.eye(2, dtype=np.complex128)
Pauli_x = np.array([[0, 1], [1, 0]], dtype=np.complex128)

# threshold
thr = 10**-9

In [111]:
def is_unitary(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    A = np.array(A)
    return np.allclose(np.eye(n), A @ A.conj().T)


def is_identity(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    return np.allclose(A, np.eye(n))


def elimination_matrix(a,b):
    # a, b allowed to be complex
    
    # impose theta real + positive {eq.10}
    theta = np.arctan(abs(b/a))
    
    # lambda is the negative arg() of a
    lamda = - np.angle(a)
    
    # {eq.12}
    mu = np.pi + np.angle(b)
    
    # {eq.7}
    U_special = np.array([ [np.exp(1j*lamda) * np.cos(theta), np.exp(1j*mu) * np.sin(theta)],
                           [-np.exp(-1j*mu) * np.sin(theta), np.exp(-1j*lamda) * np.cos(theta)] ])
    
    return U_special


def two_level_decomp(A):
    n = A.shape[0]
    decomp = []
    indices = []
    A_c = np.copy(A)

    for i in range(n-2):
        for j in range(n-1, i, -1):

            a = A_c[i,j-1]
            b = A_c[i,j]

            # --- need checks --- 
            # if A[i,j] = 0, nothing to do! Except in last row - need to check diagonal element is 1 
            if abs(A_c[i,j]) < thr:
                U_22 = Identity_22

                if j == i+1:
                    U_22 = np.array([[1 / a, 0], [0, a]])

            # if A[i,j-1] = 0, need to swap columns - again checking last row to ensure diagonal element is 1 
            elif abs(A_c[i,j-1]) < thr:
                U_22 = Pauli_x

                if j == i+1:
                    U_22 = np.array([[1 / b, 0], [0, b]])

            # Special unitary matrix
            else: 
                U_22 = elimination_matrix(a,b)

            # ----- U_22 found -----

            # multiply submatrix of A with U_22
            A_c[:,(j-1,j)] = A_c[:,(j-1,j)] @ U_22

            # If not the identity matrix - represents a gate! So should store
            if not is_identity(U_22):
                decomp.append(U_22.conj().T)
                indices.append(np.array([j-1,j]))


        # check for diagonal element equal to 1
        assert np.allclose(A_c[i,i],1.0)
    
    # lower right hand 2x2 matrix remaining after decomp
    lower_rh_matrix = A_c[n-2:n, n-2:n]
    
    # if not equal to I - is a non trivial gate
    if not is_identity(lower_rh_matrix):
        decomp.append(lower_rh_matrix)
        indices.append(np.array([n-2,n-1]))

    return decomp, indices


def gray_method(A):
    
    n = A.shape[0]
    M = np.copy(A)

    # using bitwise_xor find Gray permutations
    permutations = []
    for i in range(n):
        permutations.append(i ^ (i // 2))
        
    # 
    M[:,:] = M[:,permutations]
    M[:,:] = M[permutations,:]
    
    decomp, indices = two_level_decomp(M)
    new_ind = []
    
    decomp, indices = two_level_decomp(M)
    new_decomp = []
    new_ind = []

    for i in range(len(indices)):
        
        t = np.take(permutations, indices[i])
        if t[0]>t[1]:
            new_decomp.append(decomp[i].T)
            new_ind.append(np.sort(t))

        else:
            new_decomp.append(decomp[i])
            new_ind.append(t)
            
    return new_decomp, new_ind

    

In [108]:
n = A.shape[0]
M = np.copy(A)
permutations = []
for i in range(n):
    permutations.append(i ^ (i // 2))

# 
M[:,:] = M[:,permutations]
M[:,:] = M[permutations,:]

decomp, indices = two_level_decomp(M)
new_decomp = []
new_ind = []

for i in range(len(indices)):
#     new_ind.append(np.sort(np.take(permutations, pair, 0)))
# indices
    t = np.take(permutations, indices[i])
    if t[0]>t[1]:
        new_decomp.append(decomp[i].T)
        new_ind.append(np.sort(t))
        
    else:
        new_decomp.append(decomp[i])
        new_ind.append(t)
        
        

In [93]:
permutations

[0, 1, 3, 2]

In [80]:
class SGate:
    def __init__(self,gate,Q_id):
        self.gate = gate
        self.Q_id = Q_id

        
class CGate:
    def __init__(self,gate,Q_id):
        self.gate = gate
        self.Q_id = Q_id

        
def flip(overlay, gates):
    
    '''X gate operator determined by XOR between qbit'''
    
    Q_id = 0
    while overlay>0:
        if overlay % 2 == 1:
            gates.append(SGate(Pauli_x, Q_id))
        overlay //= 2
        Q_id += 1

        
def SpecU_2_gate(A):
    
    '''Special Unitary Gate'''
    
    u1 = A[0,0]
    u2 = A[0,1]
    theta = 2*np.arccos(abs(u1))
    lamda = np.angle(u1)
    mu = np.angle(u2)
    
    gates = []
    
    temp = lamda-mu
    if abs(temp) > thr:
        gates.append(np.diag([np.exp(0.5j * temp), np.exp(-0.5j * temp)]))
        
    if abs(theta) > thr:
        gates.append(np.array([[np.cos(theta), np.sin(theta)],
                               [-np.sin(theta), np.cos(theta)]]))
    
    temp = lamda+mu
    if abs(temp) > thr:
        gates.append(np.diag([np.exp(0.5j * temp), np.exp(-0.5j * temp)]))
    
    return gates
        
    
def U_2_gate(A):
    
    '''General Unitary Gate'''
    
    phi = np.angle(np.linalg.det(A))
    if abs(phi) < thr:
        return SpecU_2_gate(A)
    elif np.allclose(A, Pauli_x):
        return Pauli_x
    else:
        A = np.diag([1.0, np.exp(-1j * phi)]) @ A
        return SpecU_2_gate(A) + np.diag([1.0, np.exp(1j * phi)])
    
import math
def matrix_2_gates(A):
    
    '''Convert matrix into series of gates'''
    
    gates = []
    prev_flip_overlay = 0
    
    decomp, indices = gray_method(A)
    for i in range(len(decomp)):
        sub = decomp[i]
        ind1, ind2 = indices[i][0], indices[i][1]
        id_overlay = ind1 ^ ind2
        Q_id = int(math.log2(id_overlay))
        
        flip_overlay = sub.shape[0]-1 - ind2
        flip(flip_overlay ^ prev_flip_overlay, gates)
        
        for gate in U_2_gate(sub):
            gates.append(CGate(gate,Q_id))
        prev_flip_overlay = flip_overlay
        
    flip(prev_flip_overlay,gates)
    
    return(gates)
    
    
            


In [67]:
gray_method(A)

([array([[0.-0.j, 1.-0.j],
         [1.-0.j, 0.-0.j]]),
  array([[ 0.70710678-0.00000000e+00j,  0.70710678-8.65956056e-17j],
         [-0.70710678-8.65956056e-17j,  0.70710678-0.00000000e+00j]]),
  array([[ 0.57735027-0.00000000e+00j,  0.81649658-9.99919924e-17j],
         [-0.81649658-9.99919924e-17j,  0.57735027-0.00000000e+00j]]),
  array([[-7.07106781e-01+8.65956056e-17j, -5.14325540e-16-7.07106781e-01j],
         [ 5.14325540e-16-7.07106781e-01j, -7.07106781e-01-8.65956056e-17j]]),
  array([[ 0.00000000e+00+0.j,  0.00000000e+00-1.j],
         [-6.98727119e-16-1.j,  0.00000000e+00+0.j]])],
 [array([2, 3]), array([1, 3]), array([0, 1]), array([1, 3]), array([2, 3])])

In [123]:
ms = matrix_2_gates(A)
for m in ms:
    print(m.gate, m.Q_id)

[[0.59467809-0.80396391j 0.        +0.j        ]
 [0.        +0.j         0.59467809+0.80396391j]] 0
[[-0.28525208  0.95845253]
 [-0.95845253 -0.28525208]] 0
[[0.24903632+0.96849415j 0.        +0.j        ]
 [0.        +0.j         0.24903632-0.96849415j]] 0
[[0.92204237+0.38708897j 0.        +0.j        ]
 [0.        +0.j         0.92204237-0.38708897j]] 1
[[-0.52232904  0.85274403]
 [-0.85274403 -0.52232904]] 1
[[0.92204237+0.38708897j 0.        +0.j        ]
 [0.        +0.j         0.92204237-0.38708897j]] 1
[[0.83672831-0.54761824j 0.        +0.j        ]
 [0.        +0.j         0.83672831+0.54761824j]] 0
[[-0.05340704  0.99857283]
 [-0.99857283 -0.05340704]] 0
[[0.83672831-0.54761824j 0.        +0.j        ]
 [0.        +0.j         0.83672831+0.54761824j]] 0
[[0.98532638-0.17068078j 0.        +0.j        ]
 [0.        +0.j         0.98532638+0.17068078j]] 0
[[-0.09628369  0.99535393]
 [-0.99535393 -0.09628369]] 0
[[-0.48423737+0.87493666j  0.        +0.j        ]
 [ 0.        +

In [125]:
qd.quantum_decomp_main.matrix_to_gates(A)

[Rz(-2.638222449151229) on bit 0, fully controlled,
 Ry(1.860065752122905) on bit 0, fully controlled,
 Rz(1.8678622906964781) on bit 0, fully controlled,
 Rz(0.7949446655180341) on bit 1, fully controlled,
 Ry(2.1203762307453524) on bit 1, fully controlled,
 Rz(0.7949446655180348) on bit 1, fully controlled,
 X on bit 1,
 Rz(-1.1590301194041304) on bit 0, fully controlled,
 Ry(1.6242287922082768) on bit 0, fully controlled,
 Rz(-1.1590301194041308) on bit 0, fully controlled,
 X on bit 1,
 Rz(-4.152575295711639) on bit 0, fully controlled,
 Ry(1.6672294057893209) on bit 0, fully controlled,
 Rz(0.34304109058569354) on bit 0, fully controlled,
 Rz(0.1211789908330359) on bit 1, fully controlled,
 Ry(0.8474384665843779) on bit 1, fully controlled,
 Rz(0.12117899083303565) on bit 1, fully controlled,
 Rz(-0.905961578533288) on bit 0, fully controlled,
 Ry(0.6597443188399046) on bit 0, fully controlled,
 Rz(2.5074466786318603) on bit 0, fully controlled,
 R1(2.9635916657522836) on bit 0, f

In [60]:
import quantum_decomp as qd
qd.quantum_decomp_main.matrix_to_gates(A)

[X on bit 0, fully controlled,
 Ry(1.5707963267948966) on bit 1, fully controlled,
 X on bit 1,
 Ry(1.9106332362490186) on bit 0, fully controlled,
 X on bit 1,
 Rz(4.712388980384691) on bit 1, fully controlled,
 Ry(1.5707963267948961) on bit 1, fully controlled,
 Rz(1.570796326794896) on bit 1, fully controlled,
 Rz(1.5707963267948972) on bit 0, fully controlled,
 Ry(3.141592653589793) on bit 0, fully controlled,
 Rz(-1.5707963267948972) on bit 0, fully controlled]

In [11]:
# test matrix
w = np.exp((2j / 3) * np.pi)


In [41]:
A = np.array([[1, 1, 1, 0], 
                  [1, w, w * w, 0],
                  [1, w * w, w, 0], 
                  [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)
two_level_decomp(A)

([array([[ 0.70710678-0.00000000e+00j,  0.70710678-8.65956056e-17j],
         [-0.70710678-8.65956056e-17j,  0.70710678-0.00000000e+00j]]),
  array([[ 0.57735027-0.00000000e+00j,  0.81649658-9.99919924e-17j],
         [-0.81649658-9.99919924e-17j,  0.57735027-0.00000000e+00j]]),
  array([[-7.07106781e-01+8.65956056e-17j, -5.14325540e-16-7.07106781e-01j],
         [ 5.14325540e-16-7.07106781e-01j, -7.07106781e-01-8.65956056e-17j]]),
  array([[-6.98727119e-16-1.j,  0.00000000e+00+0.j],
         [ 0.00000000e+00+0.j,  0.00000000e+00-1.j]])],
 [array([1, 2]), array([0, 1]), array([1, 2]), array([2, 3])])

In [125]:
import quantum_decomp as qd

# A = np.array([[1, 1, 1, 0], 
#                   [1, w, w * w, 0],
#                   [1, w * w, w, 0], 
#                   [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)
Test = matrix_2_gates(A)

# A = np.array([[1, 1, 1, 0], 
#                   [1, w, w * w, 0],
#                   [1, w * w, w, 0], 
#                   [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)

Known = qd.quantum_decomp_main.matrix_to_gates(A)

Test[1].gate

array([[ 0.80312118,  0.59581572],
       [-0.59581572,  0.80312118]])

In [73]:
from scipy.stats import unitary_group
nq = 2
A = unitary_group.rvs(2**nq)
decomp, indices = two_level_decomp(A)
decomp

[array([[ 0.50546415+0.62219936j,  0.55400673+0.22461189j],
        [-0.55400673+0.22461189j,  0.50546415-0.62219936j]]),
 array([[ 0.34225405+3.48851895e-01j,  0.87244743-1.06843995e-16j],
        [-0.87244743-1.06843995e-16j,  0.34225405-3.48851895e-01j]]),
 array([[ 0.27534341-6.30462121e-01j,  0.72574343-8.88779365e-17j],
        [-0.72574343-8.88779365e-17j,  0.27534341+6.30462121e-01j]]),
 array([[-0.22034643+0.63506347j, -0.46381468-0.57707693j],
        [ 0.46381468-0.57707693j, -0.22034643-0.63506347j]]),
 array([[ 0.90488138+1.1019251e-01j,  0.41115362-5.0351796e-17j],
        [-0.41115362-5.0351796e-17j,  0.90488138-1.1019251e-01j]]),
 array([[-0.04388814+0.32093511j, -0.65863877-0.67916821j],
        [-0.5279768 +0.78505731j,  0.10002026+0.30809327j]])]

In [112]:
gray_method(A)

([array([[ 0.55400673+0.22461189j, -0.50546415+0.62219936j],
         [ 0.50546415+0.62219936j,  0.55400673-0.22461189j]]),
  array([[ 0.34225405+3.48851895e-01j,  0.87244743+2.80600495e-16j],
         [-0.87244743+2.80600495e-16j,  0.34225405-3.48851895e-01j]]),
  array([[ 0.27534341-6.30462121e-01j,  0.72574343-8.88779365e-17j],
         [-0.72574343-8.88779365e-17j,  0.27534341+6.30462121e-01j]]),
  array([[-0.22034643+0.63506347j, -0.46381468+0.57707693j],
         [ 0.46381468+0.57707693j, -0.22034643-0.63506347j]]),
  array([[ 0.90488138+1.1019251e-01j,  0.41115362-5.0351796e-17j],
         [-0.41115362-5.0351796e-17j,  0.90488138-1.1019251e-01j]]),
  array([[-0.5279768 +0.78505731j, -0.04388814+0.32093511j],
         [-0.10002026-0.30809327j,  0.65863877+0.67916821j]])],
 [array([2, 3]),
  array([1, 3]),
  array([0, 1]),
  array([2, 3]),
  array([1, 3]),
  array([2, 3])])

In [113]:
qd.two_level_decompose_gray(A)

[[[ 0.55400673-0.22461189j -0.50546415+0.62219936j]
  [ 0.50546415+0.62219936j  0.55400673+0.22461189j]] on (2, 3),
 [[ 0.34225405+3.48851895e-01j  0.87244743+2.80600495e-16j]
  [-0.87244743+2.80600495e-16j  0.34225405-3.48851895e-01j]] on (1, 3),
 [[ 0.27534341-6.30462121e-01j  0.72574343-8.88779365e-17j]
  [-0.72574343-8.88779365e-17j  0.27534341+6.30462121e-01j]] on (0, 1),
 [[-0.22034643-0.63506347j -0.46381468+0.57707693j]
  [ 0.46381468+0.57707693j -0.22034643+0.63506347j]] on (2, 3),
 [[ 0.90488138+1.1019251e-01j  0.41115362-5.0351796e-17j]
  [-0.41115362-5.0351796e-17j  0.90488138-1.1019251e-01j]] on (1, 3),
 [[ 0.65863877+0.67916821j -0.04388814+0.32093511j]
  [-0.10002026-0.30809327j -0.5279768 +0.78505731j]] on (2, 3)]